In [64]:
import random
import pandas as pd
import numpy as np
import os
import glob
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

def limit_range(df):
    '''
    환경 변수 별 제한 범위를 넘어서는 값을 결측치 처리
    '''
    df.loc[(df['내부온도관측치'] < 4) | (df['내부온도관측치'] > 40), '내부온도관측치'] = np.nan
    df.loc[(df['내부습도관측치'] < 0) | (df['내부습도관측치'] > 100), '내부습도관측치'] = np.nan
    df.loc[(df['co2관측치'] < 0) | (df['co2관측치'] > 1200), 'co2관측치'] = np.nan
    df.loc[(df['ec관측치'] < 0) | (df['ec관측치'] > 8), 'ec관측치'] = np.nan
    df.loc[(df['시간당분무량'] < 0) | (df['시간당분무량'] > 3000), '시간당분무량'] = np.nan
    df.loc[(df['일간누적분무량'] < 0) | (df['일간누적분무량'] > 72000), '일간누적분무량'] = np.nan
    df.loc[(df['시간당백색광량'] < 0) | (df['시간당백색광량'] > 120000), '시간당백색광량'] = np.nan
    df.loc[(df['일간누적백색광량'] < 0) | (df['일간누적백색광량'] > 2880000), '일간누적백색광량'] = np.nan
    df.loc[(df['시간당적색광량'] < 0) | (df['시간당적색광량'] > 120000), '시간당적색광량'] = np.nan
    df.loc[(df['일간누적적색광량'] < 0) | (df['일간누적적색광량'] > 2880000), '일간누적적색광량'] = np.nan
    df.loc[(df['시간당청색광량'] < 0) | (df['시간당청색광량'] > 120000), '시간당청색광량'] = np.nan
    df.loc[(df['일간누적청색광량'] < 0) | (df['일간누적청색광량'] > 2880000), '일간누적청색광량'] = np.nan
    df.loc[(df['시간당총광량'] < 0) | (df['시간당총광량'] > 120000), '시간당총광량'] = np.nan
    df.loc[(df['일간누적총광량'] < 0) | (df['일간누적총광량'] > 2880000), '일간누적총광량'] = np.nan
    return df

def time_value(df):
    ''' 
    ex) 00:59:59 => 01:00:00으로 변환 후 시간단위만 추출
    '''
    df['obs_time'] = pd.to_datetime(df["obs_time"]) + datetime.timedelta(seconds=1)
    df['obs_time'] = df['obs_time'].dt.hour
    return df


def col_cumsum(df, col, cum_col):
    '''
    시간값에 이상치가 있어서 누적값을 새로 생성
    '''
    import itertools
    df[cum_col] = 0
    for i in range(28):
        result = itertools.accumulate(df[col][i*24:(i+1)*24])
        cumsum = [value for value in result]
        df[cum_col][i*24:(i+1)*24] = cumsum
        
    return df

def make_col_data(input_path):
    df_new = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    for path in all_input_list:
        df = pd.read_csv(path)
        df = time_value(df)
        df = limit_range(df)
        df = df.fillna(method='ffill')
        df = df.iloc[:, 1:]
        df = col_cumsum(df, "시간당분무량", "일간누적분무량")
        df = col_cumsum(df, "시간당백색광량", "일간누적백색광량")
        df = col_cumsum(df, "시간당적색광량", "일간누적적색광량")
        df = col_cumsum(df, "시간당청색광량", "일간누적청색광량")
        df = col_cumsum(df, "시간당총광량", "일간누적총광량")
        col_list = df.columns[1:]
        for i in range(0,28):    
            day = df.iloc[24*i:24*i+24]
            time_list = day['obs_time'].unique()
            for col in col_list:
                for time in time_list:
                    value = day[day['obs_time']==time][col].iloc[0]
                    df[col+str(time)+'시'] = value           
            nx = df.iloc[:1, 15:]
            df_new = pd.concat([df_new, nx]).reset_index(drop=True)

    return df_new

input_path = './data/test_input/*.csv'
test = make_col_data(input_path)


C:\Users\kki96\AppData\Local\Temp\ipykernel_9956\815064419.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum
C:\Users\kki96\AppData\Local\Temp\ipykernel_9956\815064419.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+str(time)+'시'] = value
C:\Users\kki96\AppData\Local\Temp\ipykernel_9956\815064419.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get 

In [70]:
import pandas as pd

df = pd.read_csv('./data/train_input/CASE_01.csv')

In [78]:
def time_value(df):
    ''' 
    ex) 00:59:59 => 01:00:00으로 변환 후 시간단위만 추출
    '''
    df['obs_time'] = pd.to_datetime(df["obs_time"]) + datetime.timedelta(seconds=1)
    df['obs_time'] = df['obs_time'].dt.hour
    return df

In [79]:
df = time_value(df)

df.loc[(df['obs_time'] < 6), '6time'] = 1
df.loc[(df['obs_time'] >=6) & (df['base_hour'] < 12), '6time'] = 2
df.loc[(df['obs_time'] >= 12) & (df['base_hour'] < 19), '6time'] = 3
df.loc[(df['obs_time'] >= 19) & (df['base_hour'] <= 24), '6time'] = 4

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

In [74]:
pd.pivot(df, index=['DAT'], columns=['obs_time'])

내부온도관측치                                                         \
obs_time      00:00      01:00      02:00      03:00      04:00      05:00   
DAT                                                                          
0         25.300000  25.680357  25.273333  25.355000  25.391667  25.375000   
1         25.806666  25.981034  25.583334  25.476667  25.431666  25.406666   
2         26.098334  26.313559  25.878333  25.794999  25.660001  25.601667   
3         25.438333  25.783051  25.321666  25.211667  25.275000  25.245000   
4         25.470000  25.462712  25.401667  25.331667  25.461667  25.410000   
5         26.098334  26.313559  25.878333  25.794999  25.660001  25.601667   
6         25.340000  25.475000  25.373333  25.398333  25.370000  25.316667   
7         25.470000  25.462712  25.401667  25.331667  25.461667  25.410000   
8         25.340000  25.475000  25.373333  25.398333  25.370000  25.316667   
9         25.276667  25.501695  25.318333  25.283333  25.323333  25.373333   
10        25.361667  25.577586  25.308333  25.436667  25.348333  25.286667   
11        25.333333  25.391525  25.345000  25.345000  25.406666  25.356667   
12        25.290000  25.364407  25.306667  25.288333  25.298333  25.283333   
13        25.181667  25.220000  25.200000  25.226667  25.208333  25.161667   
14        25.288333  25.205000  25.248333  25.200000  25.186667  25.240000   
15        25.288333  25.266667  25.280000  25.255932  25.188333  25.260000   
16        25.158333  25.183051  25.133333  25.120000  25.128333  25.081667   
17        25.201667  25.226667  25.268333  25.240000  25.271667  25.235000   
18        25.186667  25.296610  25.215000  25.256666  25.168333  25.223333   
19        25.098333  25.122414  25.173333  25.136667  25.101667  25.180000   
20        27.086667  26.951666  26.878333  26.793333  26.698334  26.595000   
21        26.581667  26.637932  26.613334  26.598334  26.551667  26.423333   
22        26.255000  26.234483  26.125000  26.076667  25.951666  25.891666   
23        25.903333  25.896491  25.821666  25.781666  25.656667  25.570000   
24        25.905000  25.875000  25.830508  25.788333  25.683334  25.598334   
25        25.946666  25.917241  25.836666  25.773333  25.653334  25.536667   
26        26.316666  26.223729  26.121667  26.100000  26.100000  26.100000   
27        27.018333  26.965000  26.928333  26.891666  26.823333  26.793333   

                                                      ...      일간누적총광량  \
obs_time      06:00      07:00      08:00      09:00  ...        14:00   
DAT                                                   ...                
0         26.436667  26.960000  27.128334  27.366666  ...  172448.3300   
1         27.105000  27.765000  28.025000  28.196667  ...  172757.7400   
2         27.388333  28.068334  28.228333  27.823333  ...  172264.0541   
3         26.893333  27.470000  26.641667  26.379661  ...  172757.7400   
4         26.391666  26.698334  26.806666  26.938333  ...  172375.8727   
5         27.388333  28.068334  28.228333  27.823333  ...  172375.8727   
6         26.805000  27.101667  27.208334  27.799999  ...  172431.7820   
7         26.391666  26.698334  26.806666  26.938333  ...  172398.9707   
8         26.805000  27.101667  27.208334  26.711666  ...  172173.5324   
9         26.086667  26.308333  26.278333  25.585000  ...  172431.7820   
10        26.360000  26.401666  26.198333  25.921666  ...  172431.7820   
11        26.128333  26.355000  25.728333  25.405000  ...  172343.0614   
12        25.886666  26.083334  25.771667  25.288333  ...  172128.9220   
13        25.521667  25.455000  25.615000  25.693334  ...  172012.5700   
14        25.635000  25.536667  25.515000  25.448333  ...  172091.7928   
15        25.525000  25.553334  25.460000  25.443333  ...  171889.0215   
16        25.761667  25.505000  25.511667  25.505000  ...  172431.7820   
17        25.813333  25.926666  25.708334  25.690000  ...  172431.7820   
18        25.655000  25.475000  25.4533

co2관측치                                                  \
obs_time       00:00       01:00       02:00       03:00       04:00   
DAT                                                                    
0         536.016667  528.696429  532.833333  545.566667  558.583333   
1         523.216667  531.844828  540.633333  548.516667  555.633333   
2         516.416667  518.084746  527.733333  537.600000  543.983333   
3         531.450000  536.474576  547.833333  557.850000  559.950000   
4         543.600000  548.491525  553.416667  558.766667  564.083333   
5         516.416667  518.084746  527.733333  537.600000  543.983333   
6         543.533333  548.533333  556.233333  561.383333  564.933333   
7         543.600000  548.491525  553.416667  558.766667  564.083333   
8         543.533333  548.533333  556.233333  561.383333  564.933333   
9         544.516667  539.915254  541.166667  543.066667  544.600000   
10        531.333333  534.035088  540.250000  544.483333  549.316667   
11        705.083333  689.637931  679.083333  670.133333  666.650000   
12        565.166667  571.932203  578.133333  576.700000  575.733333   
13        563.283333  564.650000  564.800000  566.150000  568.033333   
14        552.300000  553.050000  556.016667  550.450000  545.916667   
15        577.700000  589.100000  596.000000  602.716667  609.183333   
16        587.983333  586.338983  588.183333  590.416667  593.466667   
17        594.783333  587.700000  581.316667  575.733333  572.850000   
18        597.633333  595.762712  593.383333  592.950000  590.483333   
19        642.483333  630.948276  614.150000  607.216667  601.300000   
20        589.500000  584.700000  580.283333  575.166667  573.816667   
21        544.833333  549.482759  557.183333  566.650000  577.516667   
22        555.783333  563.344828  559.783333  555.783333  554.833333   
23        559.600000  557.666667  552.016667  551.166667  548.700000   
24        542.916667  540.433333  539.389831  538.633333  537.516667   
25        585.266667  589.413793  586.200000  588.316667  590.900000   
26        724.883333  700.949152  684.500000  667.916667  652.400000   
27        778.100000  746.366667  718.533333  695.500000  679.750000   

                                                                      ...  \
obs_time       05:00       06:00       07:00       08:00       09:00  ...   
DAT                                                                   ...   
0         571.483333  575.083333  574.550000  570.500000  568.100000  ...   
1         563.700000  565.166667  564.233333  557.366667  546.050000  ...   
2         551.066667  544.116667  536.300000  539.666667  535.933333  ...   
3         562.916667  553.400000  546.666667  547.200000  566.483333  ...   
4         568.466667  565.883333  565.316667  564.433333  559.700000  ...   
5         551.066667  544.116667  536.300000  539.666667  535.933333  ...   
6         569.916667  562.350000  559.416667  563.133333  552.000000  ...   
7         568.466667  565.883333  565.316667  564.433333  559.700000  ...   
8         569.916667  562.350000  559.416667  563.133333  564.666667  ...   
9         543.033333  532.800000  528.033333  537.116667  560.333333  ...   
10        557.500000  551.416667  546.050000  552.750000  577.400000  ...   
11        666.700000  652.966667  636.116667  629.183333  616.083333  ...   
12        575.683333  563.400000  554.850000  564.116667  574.283333  ...   
13        569.850000  553.116667  540.883333  539.716667  538.150000  ...   
14        544.966667  520.233333  501.366667  498.483333  496.716667  ...   
15        615.683333  586.283333  561.500000  566.700000  589.000000  ...   
16        596.183333  558.950000  532.066667  622.033333  639.583333  ...   
17        567.083333  514.366667  476.866667  488.266667  512.250000  ...   
18        589.366667  528.966667  486.350000  492.833333  561.983333  ...   
19        596.300000  528.050000  489.983333  545.166667  550.333333  ...   
20        